<a href="https://colab.research.google.com/github/iannwtf19/iannwtf19/blob/homework10-nlp/Homework10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import re
import os 
from google.colab import drive

In [ ]:
drive.mount("/content/drive", force_remount=True)
os.chdir("drive/MyDrive/tensorflow")

#text_file = "drive/MyDrive/tensorflow/bible.txt"

Mounted at /content/drive


In [ ]:
text_file = open("bible.txt")
text_file = text_file.read()

In [ ]:
word_sequence = tf.keras.preprocessing.text.text_to_word_sequence(text_file)

In [ ]:
print(word_sequence[:10])

['the', 'first', 'book', 'of', 'moses', 'called', 'genesis', '1', '1', 'in']


In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=10000,
    #filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    #oov_token=None,
    analyzer=None
)
tokenizer.fit_on_texts([text_file])
vocab = tokenizer.word_index

#  len_vocab = len(vocab)
inverse_vocab = {index: token for token, index in vocab.items()}

tokens = tf.keras.preprocessing.text.text_to_word_sequence(text_file)
sequence = tokenizer.texts_to_sequences(tokens)

In [ ]:
print(vocab)
print(len(vocab))
print(sequence[:10])
print(tokens[:10])

{'the': 1, 'and': 2, 'of': 3, 'to': 4, 'that': 5, 'in': 6, 'he': 7, 'shall': 8, 'unto': 9, 'for': 10, 'i': 11, 'his': 12, 'a': 13, 'lord': 14, 'they': 15, 'be': 16, 'is': 17, 'him': 18, 'not': 19, 'them': 20, 'it': 21, 'with': 22, 'all': 23, 'thou': 24, 'thy': 25, 'was': 26, 'god': 27, 'which': 28, 'my': 29, 'me': 30, 'said': 31, 'but': 32, 'ye': 33, 'their': 34, 'have': 35, 'will': 36, 'thee': 37, 'from': 38, 'as': 39, 'are': 40, 'when': 41, 'this': 42, '1': 43, 'out': 44, 'were': 45, 'upon': 46, '2': 47, 'by': 48, 'you': 49, 'man': 50, 'israel': 51, '3': 52, '4': 53, 'up': 54, 'son': 55, '7': 56, '5': 57, 'there': 58, '6': 59, 'hath': 60, 'king': 61, '9': 62, '8': 63, 'then': 64, 'people': 65, '10': 66, '11': 67, 'came': 68, 'had': 69, 'house': 70, 'into': 71, 'on': 72, 'her': 73, 'come': 74, '12': 75, 'one': 76, '13': 77, '14': 78, 'we': 79, 'children': 80, 'before': 81, '15': 82, 'your': 83, 'also': 84, 'day': 85, 'land': 86, 'an': 87, '16': 88, 'so': 89, 'against': 90, 'men': 91, 

In [ ]:
window_size = 4
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      sequence,
      vocabulary_size=len(sequence),
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))
print(positive_skip_grams[0])

6770366
[[2704], [53]]


In [ ]:
for target, context in positive_skip_grams[:100]:
  print(f"({target[0]}, {context[0]}): ({inverse_vocab[target[0]]}, {inverse_vocab[context[0]]})")

In [ ]:
for i in range(10):
  print(positive_skip_grams[i])

In [ ]:
# Negative sampling
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=len(vocab),  # pick index of the samples from [0, vocab_size]
    seed=42,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)

In [ ]:
import numpy as np
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([ 348   30 2104  101], shape=(4,), dtype=int64)
['mighty', 'me', 'amalekites', '20']
